In [1]:
! pip install stanza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 38.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import json
import sqlite3

In [3]:
import stanza
stanza.download('ru')

stnz = stanza.Pipeline('ru', use_gpu = True)

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: ru (Russian) ...


INFO:stanza:Downloaded file to /root/stanza_resources/ru/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| pos       | syntagrus_charlm   |
| lemma     | syntagrus_nocharlm |
| depparse  | syntagrus_charlm   |
| ner       | wikiner            |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
/usr/local/lib/python3.10/dist-packages/stanza/models/tokenization/trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed d

In [4]:
from tqdm import tqdm

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


переменные

In [6]:
file_name = 'dota.json'
theme_name = 'Чат игроков'

Извлечение последних индексов

In [7]:
conn = sqlite3.connect("/content/drive/MyDrive/coco/данные_обработанные/data_pis_ko.db") #путь
c = conn.cursor()

In [8]:
c.execute("select * from sqlite_master where type='table'")

In [9]:
c.fetchall()

[('table',
  'authors',
  'authors',
  46354,
  'CREATE TABLE "authors" (\n\t"index"\tINTEGER,\n\t"author_id"\tINTEGER,\n\t"author_tg_id"\tTEXT,\n\tPRIMARY KEY("author_id")\n)'),
 ('table',
  'communities',
  'communities',
  2256,
  'CREATE TABLE "communities" (\n\t"index"\tINTEGER,\n\t"community_id"\tINTEGER,\n\t"community_name"\tTEXT,\n\tPRIMARY KEY("community_id")\n)'),
 ('table',
  'morphs',
  'morphs',
  22242,
  'CREATE TABLE "morphs" (\n\t"index"\tINTEGER,\n\t"id"\tINTEGER,\n\t"text"\tTEXT,\n\t"lemma"\tTEXT,\n\t"upos"\tTEXT,\n\t"head"\tINTEGER,\n\t"deprel"\tTEXT,\n\t"start_char"\tINTEGER,\n\t"end_char"\tINTEGER,\n\t"ner"\tTEXT,\n\t"message_id"\tINTEGER,\n\t"feats"\tTEXT,\n\t"misc"\tTEXT,\n\tPRIMARY KEY("id")\n)'),
 ('table',
  'sourses',
  'sourses',
  36896,
  'CREATE TABLE "sourses" (\n\t"index"\tINTEGER,\n\t"sourse_id"\tINTEGER,\n\t"sourse_name"\tTEXT,\n\tPRIMARY KEY("sourse_id")\n)'),
 ('table',
  'texts',
  'texts',
  399,
  'CREATE TABLE "texts" (\n\t"index"\tINTEGER,\n\t

In [10]:
c.execute("select count(*) from authors")
author_id = list(c)[0][0]

In [11]:
author_id

25436

In [12]:
c.execute("select count(*) from morphs")
word_id = list(c)[0][0]

In [13]:
word_id

8248909

In [14]:
c.execute("select count(*) from sourses")
sourse_id = list(c)[0][0]

In [15]:
sourse_id

1

In [16]:
c.execute("select count(*) from texts")
message_id = list(c)[0][0]

In [17]:
message_id

1066288

In [18]:
c.execute("select count(*) from communities")
community_id = list(c)[0][0]

In [19]:
community_id

11

In [20]:
c.execute("select count(*) from themes")
theme_id = list(c)[0][0]

In [21]:
theme_id

4

Обработка

In [22]:
with open(f'/content/drive/MyDrive/coco/данные_сырые/{file_name}', 'r', encoding = 'utf-8') as res:
    tg_conv = json.load(res)

In [23]:
len(tg_conv['messages'])

839370

In [24]:
def morph(text, message_id):
  global word_id
  analys = stnz(text).to_dict()
  for sent in analys:
    for token in sent:
      token['id'] = word_id
      token['message_id'] = message_id
      tagging.append(token)
      word_id += 1

In [25]:
# все листы словарей
tagging = []
text = [] #[{'message_id': , 'message_tg_id': , 'text': }, ...]
metadata = []
author = []
sourse = []
community = []
theme = []
thrash = []

tg_to_local_id = {}
tg_to_local_author = {}

In [26]:
"""# загружаем словари соответсвий индексов из этого же чата
with open('/content/drive/MyDrive/coco/данные_обработанные/tg_to_local_id.json', 'r') as file:
    tg_to_local_id = json.load(file)
with open('/content/drive/MyDrive/coco/данные_обработанные/tg_to_local_author.json', 'r') as file:
    tg_to_local_author = json.load(file)"""

"# загружаем словари соответсвий индексов из этого же чата\nwith open('/content/drive/MyDrive/coco/данные_обработанные/tg_to_local_id.json', 'r') as file:\n    tg_to_local_id = json.load(file)\nwith open('/content/drive/MyDrive/coco/данные_обработанные/tg_to_local_author.json', 'r') as file:\n    tg_to_local_author = json.load(file)"

In [27]:
len(tg_to_local_id)

0

In [28]:
len(tg_to_local_author)

0

In [29]:
c.execute("select * from authors")
new_list = c.fetchall()
print(len(new_list))
print(len(set([i[2] for i in new_list])))

25436
25399


In [30]:
sourse.append({'sourse_id': 0, 'sourse_name': 'tg'})
community.append({'community_id': community_id, 'community_name': tg_conv['name']})
theme.append({'theme_id': 3, 'theme_name': theme_name}) #поправить этот блок -- сейчас, ручное введение (тема и сорс)

for message in tqdm(tg_conv['messages']):
    if message['text_entities']:
        text_list = message['text_entities'][0]
        if text_list['type'] == 'plain':
            if len(text_list['text']) > 1:

                morph(text_list['text'], message_id)
                text.append({'message_id': message_id,
                             'text': text_list['text']})
                tg_to_local_id[message['id']] = message_id

                # проверка, что автор уже был
                cur_author_id = tg_to_local_author.get(message['from_id'])
                if not cur_author_id:
                    tg_to_local_author[message['from_id']] = author_id
                    author.append({'author_id': author_id,
                                  'author_tg_id': message['from_id']})

                    cur_author_id = author_id
                    author_id += 1

                reply_tg_id = message.get('reply_to_message_id')
                reply_local_id = tg_to_local_id.get(reply_tg_id)

                metadata.append({'message_id': message_id,
                'author_id': cur_author_id,
                'sourse_id': sourse_id,
                'community_id': community_id,
                'theme_id': theme_id,
                'date': message['date'],
                'reply_to_id': reply_local_id})

                message_id += 1

            # сохраняем все, что не пршло по условиям
            else: thrash.append(message)
        else: thrash.append(message)
    else: thrash.append(message)

  1%|          | 4482/839370 [04:16<13:14:52, 17.51it/s]


KeyboardInterrupt: 

In [31]:
len(text)

4064

In [32]:
len(author)

313

In [33]:
len(tg_to_local_author)

313

In [34]:
# мусор на дозапись
with open('/content/drive/MyDrive/coco/данные_обработанные/thrash.json', 'r', encoding = 'utf-8') as old:
    old_thrash = json.load(old)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [35]:
old_thrash += thrash

NameError: name 'old_thrash' is not defined

In [36]:
with open('/content/drive/MyDrive/coco/данные_обработанные/thrash.json', 'w') as file:
    json.dump(old_thrash, file)

NameError: name 'old_thrash' is not defined

In [37]:
"""# сохраняем словари соответсвий индексов, чтобы потом использовать при дозаписи из того же чата
with open('/content/drive/My Drive/писько/данные_обработанные/tg_to_local_id.json', 'w') as file:
    json.dump(tg_to_local_id, file)

with open('/content/drive/My Drive/писько/данные_обработанные/tg_to_local_author.json', 'w') as file:
    json.dump(tg_to_local_author, file)"""

"# сохраняем словари соответсвий индексов, чтобы потом использовать при дозаписи из того же чата\nwith open('/content/drive/My Drive/писько/данные_обработанные/tg_to_local_id.json', 'w') as file:\n    json.dump(tg_to_local_id, file)\n\nwith open('/content/drive/My Drive/писько/данные_обработанные/tg_to_local_author.json', 'w') as file:\n    json.dump(tg_to_local_author, file)"

Дозаписываем в таблицу

In [38]:
af = pd.DataFrame(author)
af

,author_id,author_tg_id
0,25436,user925451535
1,25437,user727582225
2,25438,user5516463324
3,25439,user661987510
4,25440,user2976881
...,...,...
308,25744,user855082501
309,25745,user1747347003
310,25746,user578062353
311,25747,user394178400


In [39]:
af = pd.DataFrame(author)
af.to_sql("authors",conn, if_exists='append')

313

In [40]:
mf = pd.DataFrame(metadata)
mf.to_sql("metadata",conn, if_exists='append')

4064

In [41]:
tf = pd.DataFrame(text)
tf.to_sql("texts",conn, if_exists='append')

4064

In [42]:
sf = pd.DataFrame(sourse)
sf.to_sql("sourses",conn, if_exists='append')

IntegrityError: UNIQUE constraint failed: sourses.sourse_id

In [43]:
mp = pd.DataFrame(tagging).loc[:, ['id', 'text', 'lemma', 'upos',	'head',	'deprel',
                                    'start_char',	'end_char',	'ner',
                                    'message_id',	'feats',	'misc']]
mp.to_sql("morphs", conn, if_exists='append')

29059

In [44]:
thf = pd.DataFrame(theme)
thf.to_sql("themes",conn, if_exists='append')

1

In [45]:
cf = pd.DataFrame(community)
cf.to_sql("communities",conn, if_exists='append')

1